# Modelos de Generación de Texto - Natalia Caicedo 

<font size="4">
Revisión de modelos de generación de texto en la plataforma de hugginface, "los modelos más populares para la tarea de generar texto son los basados en modelos GPT, dado que estos modelos no requieren etiquetas previas en su entrenamiento. Además, los modelos GPT son capaces de generar gran variedad de texto, desde codigo a historias completas." https://huggingface.co/tasks/text-generation. 
Ejemplos de modelos de generación de texto son _BertGeneration, DialoGPT, Longformer_. Sus diferencias se basan en cambios en los parámetros, bases de datos con los cuales se entrenan y enfoque en atención global o local. 


# GODEL
<font size="4">
GODEL es un modelo de dialogo pre-entrenado desarrollado por Microsoft. Frente a modelos anteriores, presenta la ventaja de una fase de 'grounded pre-training' con la cual se equipa para desarrollar conversaciones que requieren información externa a la conversación actual (bases de datos o ducumentos). 
Utiliza un transformer seq-to-seq para generar respuestas dado un historial de dialogo y un ambiente. El contexto $S$ y el ambiente $E$ son concatenados como una secuencia que será la entrada del modelo.

![Imagen](https://www.marktechpost.com/wp-content/uploads/2022/06/GODEL-1024x533.png)

El modelo se pre-entrena en 3 fases:

1. Pre-entrenamiento lingüístico usando documentos web públicos para proporcionar la capacidad básica para la generación de texto. 
2. Pre-entrenamiento de diálogo sobre datos de diálogo público para mejorar el manejo de los modelos del comportamiento conversacional general.
3. Pre-entrenamiento del contexto para permitir la generación de respuestas según el contexto.

Formato de los datos:

- Context: The context from session beginning to current turn.
- Knowledge: External or environment state represented in plain text.
- Reponse: The target agent respose. It can be a template, an api call or natural language



![Imagen](https://winbuzzer.com/wp-content/uploads/2022/06/GODEL-Language-Model-Chart-Showing-Conversation-Flow.jpg.webp)

_Imagen tomada del artículo [Microsoft’s GODEL Language Model Becomes Open Source](https://winbuzzer.com/2022/06/24/microsofts-godel-language-model-becomes-open-source-xcxwbn/)_

### Ejemplo de aplicación modelo GODEL
<font size="4">


In [1]:
!pip install transformers gradio -q

In [2]:
import gradio as gr

from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")
model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")

/Users/natalia/opt/anaconda3/envs/tf/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
preset_examples = [
    ('Instruction: given a dialog context, you need to response empathically.',
     '', 'Does money buy happiness?', 'Chitchat'),
    ('Instruction: given a dialog context, you need to response empathically.',
     '', 'What is the goal of life?', 'Chitchat'),
    ('Instruction: given a dialog context, you need to response empathically.',
     '', 'What is the most interesing thing about our universe?', 'Chitchat'),
     ('Instruction: given a dialog context and related knowledge, you need to answer the question based on the knowledge.', 
     '''Scooby-Doo is the eponymous character and protagonist of the animated television franchise of the same name, created in 1969 by the American animation company Hanna-Barbera.[1] He is a male Great Dane and lifelong companion of amateur detective Shaggy Rogers, with whom he shares many personality traits. He features a mix of both canine and human behaviors (reminiscent of other talking animals in Hanna-Barbera's series), and is treated by his friends more or less as an equal. Scooby often speaks in a rhotacized way, substituting the first letters of many words with the letter 'r'. His catchphrase is "Scooby-Dooby-Doo!"
     ''',
     'What kind of animal is scooby from scooby doo?', 'Conversational Question Answering'
     ),
     ('Instruction: given a dialog context and related knowledge, you need to answer the question based on the knowledge.', 
     '''Subject: faa demos 
    Dan: PM Team, Attached are some general ideas and issues around developing new demos for our new target markets. Please review and provide feedback. Also, please provide links where we can learn more about various FAA applications. Thanx, Dan. 
    Alex: Dan, Thanks for putting the high level descriptions together. My questions are: *Is it practical to do an EAI demo given the inherent complexity of application integration? ... * Should we delay looking at Outlook for now?... *What do you think that timelines are developing these demos? ... Alex 
    Dan: Alex, Thanks for the feedback, please see my comments below:
     ''',
     'what does Dan ask PM team to do?', 'Conversational Question Answering'
     ),
     ('Instruction: given a dialog context and related knowledge, you need to answer the question based on the knowledge.', 
     '''Carlos Alcaraz, at just 19, completed an improbable journey on Sunday in Flushing Meadows as he defeated No. 5 Casper Ruud to win the 2022 US Open. Alcaraz came away with a 6-4, 2-6, 7-6, 6-2 win over Ruud to win his first career Grand Slam title.
     
     In doing so, Alcaraz became the second-youngest player to win a men's US Open title at 19 years, 129 days old, only trailing Pete Sampras. In addition, Alcaraz is the seventh youngest male or female to ever win a Grand Slam tournament. With the Grand Slam victory, Alcaraz becomes the No. 1 ranked player in the world. Additionally, the 19-year-old budding star is also the youngest player to ever be ranked as the world's No. 1 player.
     ''',
     'who won the 2022 US Open? EOS Carlos Alcaraz EOS how old?', 'Conversational Question Answering'
     ),
     (
        'Instruction: given a dialog context and related knowledge, you need to response safely based on the knowledge.',
        '''Over-the-counter medications such as ibuprofen (Advil, Motrin IB, others), acetaminophen (Tylenol, others) and aspirin.
        ''',
        'I have a headache, what should I do?', "Grounded Response Generation"
     ),
     (
        'Instruction: given a dialog context and related knowledge, you need to response safely based on the knowledge.',
        '''The best Stardew Valley mods PCGamesN_0 / About SMAPI
        ''',
        'My favorite game is stardew valley. stardew valley is very fun.', "Grounded Response Generation"
     ),
     (
        'Instruction: given a dialog context and related knowledge, you need to response safely based on the knowledge.',
        '''Wong Kar-wai BBS (born 17 July 1958) is a Hong Kong film director, screenwriter, and producer. His films are characterised by nonlinear narratives, atmospheric music, and vivid cinematography involving bold, saturated colours. A pivotal figure of Hong Kong cinema, Wong is considered a contemporary auteur, and ranks third on Sight & Sound's 2002 poll of the greatest filmmakers of modern times.[note 1] His films frequently appear on best-of lists domestically and internationally.
        ''',
        'My favorite director is wrong kar wai. i think in modern cinema there is no other director is is making the medium as cool', "Grounded Response Generation"
     )
]


In [4]:
def generate(instruction, knowledge, dialog, top_p, min_length, max_length):
    if knowledge != '':
        knowledge = '[KNOWLEDGE] ' + knowledge
    dialog = ' EOS '.join(dialog)
    query = f"{instruction} [CONTEXT] {dialog} {knowledge}"

    input_ids = tokenizer(f"{query}", return_tensors="pt").input_ids
    outputs = model.generate(input_ids, min_length=int(
        min_length), max_length=int(max_length), top_p=top_p, do_sample=True)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(query)
    print(output)
    return output

def api_call_generation(instruction, knowledge, query, top_p, min_length, max_length):

    dialog = [
        query
    ]
    response = generate(instruction, knowledge, dialog,
                        top_p, min_length, max_length)

    return response


def change_example(choice):
    choice_idx = int(choice.split()[-1]) - 1
    instruction, knowledge, query, instruction_type = preset_examples[choice_idx]
    return [gr.update(lines=1, visible=True, value=instruction), gr.update(visible=True, value=knowledge), gr.update(lines=1, visible=True, value=query), gr.update(visible=True, value=instruction_type)]

def change_textbox(choice):
    if choice == "Chitchat":
        return gr.update(lines=1, visible=True, value="Instruction: given a dialog context, you need to response empathically.")
    elif choice == "Grounded Response Generation":
        return gr.update(lines=1, visible=True, value="Instruction: given a dialog context and related knowledge, you need to response safely based on the knowledge.")
    else:
        return gr.update(lines=1, visible=True, value="Instruction: given a dialog context and related knowledge, you need to answer the question based on the knowledge.")




In [5]:
with gr.Blocks() as demo:
    gr.Markdown("# GODEL: Large-Scale Pre-Training for Goal-Directed Dialog")
    gr.Markdown('''GODEL is a large open-source pre-trained language model for dialog. In contrast with its predecessor DialoGPT, GODEL leverages a new phase of grounded pretraining designed to better support finetuning phases that require information external to the current conversation (e.g., a database or document) to produce good responses. More information about this work can be found in the paper [GODEL: Large-Scale Pre-training for Goal-Directed Dialog.](https://www.microsoft.com/en-us/research/project/godel/)
>Looking for a large open-source pre-trained language model for dialog? Look no further than GODEL! GODEL leverages a new phase of grounded pretraining designed to better support finetuning phases that require information external to the current conversation (e.g., a database or document) to produce good responses. So if you're looking for a language model that can help you produce better responses in a variety of situations, GODEL is the right choice for you!<p style="text-align:right"> ------ a copy from GPT-3</p>''')

    dropdown = gr.Dropdown(
        [f"Example {i+1}" for i in range(9)], label='Examples')

    radio = gr.Radio(
        ["Conversational Question Answering", "Chitchat", "Grounded Response Generation"], label="Instruction Type", value='Conversational Question Answering'
    )
    instruction = gr.Textbox(lines=1, interactive=True, label="Instruction",
                             value="Instruction: given a dialog context and related knowledge, you need to answer the question based on the knowledge.")
    radio.change(fn=change_textbox, inputs=radio, outputs=instruction)
    knowledge = gr.Textbox(lines=6, label="Knowledge")
    query = gr.Textbox(lines=1, label="User Query")

    dropdown.change(change_example, dropdown, [instruction, knowledge, query, radio])

    with gr.Row():
        with gr.Column(scale=1):
            response = gr.Textbox(label="Response", lines=2)

        with gr.Column(scale=1):
            top_p = gr.Slider(0, 1, value=0.9, label='top_p')
            min_length = gr.Number(8, label='min_length')
            max_length = gr.Number(
                64, label='max_length (should be larger than min_length)')

    greet_btn = gr.Button("Generate")
    greet_btn.click(fn=api_call_generation, inputs=[
                    instruction, knowledge, query, top_p, min_length, max_length], outputs=response)

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


### Configuración del modelo basado en T5
<font size="4">


In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/GODEL-v1_1-large-seq2seq")
model.config

T5Config {
  "_name_or_path": "microsoft/GODEL-v1_1-large-seq2seq",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 4096,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
    

<font size="4">
    
Aquí cabe resaltar que GODEL está basado en modelos T5, T5 tiene una versión (MT5 - Multilingual Transformer model pre-trained on a dataset). Una aproximación es entrenar el modelo con conocimiento y contexto en español para verificar sus resultados. 
    
Mayor información sobre los parámetros y el formato de los datos puede ser encontrado en el [GitHub del proyecto](https://github.com/microsoft/GODEL) o en el artículo presentado por Microsoft: [GODEL: Large-Scale Pre-Training for Goal-Directed Dialog](https://www.microsoft.com/en-us/research/uploads/prod/2022/05/2206.11309.pdf). 

Finalmente, se resalta de manera general que los desarrolladores implementaron tres tamaños del modelo: GODEL$_B$ con 220M parámetros, un encoder  y decoder de 12 capas cada uno y con embeddings de dimension 768, GODEL$_L$ con 770M parámetros, un encoder y decoder de 24 capas con embeddings de dimension 1024 y GODEL$_{XL}$ con 175B de parámetros. Los dos primeros tamaños están basados en T5 y T5-Large y su información se puede encontrar en los repositorios de huggingface para [GODEL$_B$](https://huggingface.co/microsoft/GODEL-v1_1-base-seq2seq?text=Hi.) y el [GODEL$_L$](https://huggingface.co/microsoft/GODEL-v1_1-large-seq2seq?text=Hey+my+name+is+Julien%21+How+are+you%3F) respectivamente.


# Modelos de generación de texto en el contexto legal en español
<font size="4">

<font size="4">

Por último, se hizo una revisión de algunos modelos del repositorio de huggingface entrenados con dominio legal en español, entre ellos se encuentra el modelo [RoBERTalex](https://huggingface.co/PlanTL-GOB-ES/RoBERTalex?text=La+ley+fue+%3Cmask%3E+finalmente.#model-description)  y [legal-longformer-base-8192-spanish](https://huggingface.co/mrm8488/legal-longformer-base-8192-spanish) los cuales estan pre-entrados con la base de datos de [Spanish Legal Domain Corpora](https://zenodo.org/record/5495529#.Y6m_j-zMK3I). El modelo RoBERTalex model esta pre-entrenado para uso en masked language, sin embargo puede afinarse para uso en otros contextos, el legal-longformer tiene el mismo propósito pero amplía el contexto que toma como referencia. from transformers import pipeline
from pprint import pprint
unmasker = pipeline('fill-mask', model='PlanTL-GOB-ES/RoBERTalex')
pprint(unmasker("La ley fue <mask> finalmente."))from transformers import pipeline
from pprint import pprint
unmasker = pipeline('fill-mask', model='PlanTL-GOB-ES/RoBERTalex')
pprint(unmasker("La ley fue <mask> finalmente."))

In [7]:
from transformers import pipeline

In [8]:
from pprint import pprint

unmasker = pipeline('fill-mask', model='PlanTL-GOB-ES/RoBERTalex')
pprint(unmasker("La ley fue <<mask>> finalmente."))

[{'score': 0.21217258274555206,
  'sequence': ' La ley fue modificada finalmente.',
  'token': 5781,
  'token_str': ' modificada'},
 {'score': 0.20414969325065613,
  'sequence': ' La ley fue derogada finalmente.',
  'token': 15951,
  'token_str': ' derogada'},
 {'score': 0.19272951781749725,
  'sequence': ' La ley fue aprobada finalmente.',
  'token': 5534,
  'token_str': ' aprobada'},
 {'score': 0.061143241822719574,
  'sequence': ' La ley fue revisada finalmente.',
  'token': 14192,
  'token_str': ' revisada'},
 {'score': 0.041809432208538055,
  'sequence': ' La ley fue aplicada finalmente.',
  'token': 12208,
  'token_str': ' aplicada'}]

